In [1]:
# # for colab compatibility
# !pip install shap
# !pip install ppscore
# !pip install xgboost
# !pip install lightgbm

In [2]:
# %run data_load_wrapper.ipynb

In [51]:
from tqdm.notebook import trange
import sklearn.datasets as data
import pandas as pd
from sklearn.inspection import permutation_importance
from sklearn import *
from sklearn.feature_selection import *
from sklearn.datasets import make_classification
from sklearn.utils import Bunch
import matplotlib as plt
import seaborn as sns
from boruta import BorutaPy
import shap
import numpy as np
import ppscore as pps
from datetime import datetime
import lightgbm as lgbm
import xgboost

In [72]:
from heapq import nlargest

def sklearn_to_df(dataset, with_target=True):
    df = pd.DataFrame(dataset.data, columns=dataset.feature_names)
    y = dataset.target
    if with_target == True:
        df = df.join(pd.DataFrame(y, columns=['target']), on=df.index)
        return df 
    else:
        return df, y

def get_feat_imp_data_based(imp_type, x_train, y_train):
    imp_type_dict = {
        'mutual_info' : 'mutual_info_classif(x_train, y_train)',
        'f_test' : 'f_classif(x_train, y_train)[0]',       
        'pps' : "pps.predictors(x_train.join(y_train), 'target', sorted=False, output='df').model_score",
        'pearson' : "abs(x_train.join(y_train).corr(method='pearson').target).drop('target', axis=0)",
        'spearman': "abs(x_train.join(y_train).corr(method='spearman').target).drop('target', axis=0)",
        'kendall' : "abs(x_train.join(y_train).corr(method='kendall').target).drop('target', axis=0)",
    
    }
    timings = %timeit -o -n1 -r10 eval(imp_type_dict.get(imp_type))
    timings = np.array(timings.timings)
    return eval(imp_type_dict.get(imp_type)), timings

def get_feat_imp_model_based(imp_type, m, x_train, y_train):
    imp_type_dict = {
        'gini' : "m.feature_importances_",
        'pimp' : f"""permutation_importance(m, x_train, y_train, n_repeats=5, 
        random_state=0).importances_mean""",
        'shap' : "abs(np.array(shap.TreeExplainer(m).shap_values(x_train))).sum(axis=0)"
    }
    timings = %timeit -o -n1 -r10 eval(imp_type_dict.get(imp_type))
    timings = np.array(timings.timings)
    return eval(imp_type_dict.get(imp_type)), timings

def get_syn_stats_data_based(imp_types, datasets, dataset_states, 
                             informative_pct):
    results = []
    datasets_str = list(datasets.keys())
    for j in trange(len(datasets_str), desc=f'Dataset progress'):
        for i in trange(len(dataset_states), desc=f'Processing {datasets_str[j]}'):
            for k in trange(len(imp_types)):
                imp_type = imp_types[k] 
                dataset_state = dataset_states[i]
                dataset_str = datasets_str[j]
                dataset = datasets.get(datasets_str[j])
                df = sklearn_to_df(dataset)
                x_train, x_test, y_train, y_test = model_selection.train_test_split(df,
                                                                   dataset.target, 
                                                                   test_size=0.3, 
                                                                   random_state=dataset_state,
                                                                   stratify=df.target
                                                                  )
                y_train = x_train.target
                x_train = x_train.drop('target', axis=1)
                feat_size = len(x_train.columns)
                sel_feat_size = int(feat_size * informative_pct)
                fi, timings = get_feat_imp_data_based(imp_type, x_train, y_train)
                fi_order = nlargest(feat_size, range(len(fi)), key=lambda idx: fi[idx])
                fi_selected = fi_order[:sel_feat_size]
                result = [dataset_str, imp_type, 0, dataset_state, fi_order, fi_selected, timings]
#                 print(result)
                results.append(result)
#                 print(results)
        columns = ['dataset', 'imp_type', 'random_state', 'data_split', 
                       'fi_order', 'fi_selected', 'timings']
        res = pd.DataFrame(results, columns=columns)
    return res
        
def get_syn_stats_model_based(imp_types, datasets, 
                              dataset_states, informative_pct):
    results = []
    datasets_str = list(datasets.keys())
    for j in trange(len(datasets_str), desc=f'Dataset progress'):
        for i in trange(len(dataset_states), desc=f'Processing {datasets_str[j]}'):
            for k in trange(len(imp_types)):
                imp_type = imp_types[k] 
                dataset_state = dataset_states[i]
                dataset_str = datasets_str[j]
                dataset = datasets.get(datasets_str[j])
                df = sklearn_to_df(dataset, with_target=True)
                x_train, x_test, y_train, y_test = model_selection.train_test_split(df,
                                                                   dataset.target, 
                                                                   test_size=0.3, 
                                                                   random_state=dataset_state,
                                                                   stratify=df.target
                                                                  )
                y_train = x_train.target
                x_train = x_train.drop('target', axis=1)
                feat_size = len(x_train.columns)
                sel_feat_size = int(feat_size * informative_pct)
                m = ensemble.RandomForestClassifier(random_state=0, n_jobs=-1)
                m.fit(x_train, y_train)
                fi, timings = get_feat_imp_model_based(imp_type, m, x_train, y_train)
                fi_order = nlargest(feat_size, range(len(fi)), key=lambda idx: fi[idx])
                fi_selected = fi_order[:sel_feat_size]
                result = [dataset_str, imp_type, 0, dataset_state, fi_order, fi_selected, timings]
#                 print(result)
                results.append(result)
#                 print(results)
        columns = ['dataset', 'imp_type', 'random_state', 'data_split', 
                       'fi_order', 'fi_selected', 'timings']
        res = pd.DataFrame(results, columns=columns)
    return res


from wrapt_timeout_decorator import *

@timeout(10)
def get_shap_fi(imp_type, dataset_state, dataset_str, dataset, informative_pct):
    
    result = []
    df = sklearn_to_df(dataset, with_target=True)
    x_train, x_test, y_train, y_test = model_selection.train_test_split(df,
                                                       dataset.target, 
                                                       test_size=0.3, 
                                                       random_state=dataset_state,
                                                       stratify=df.target
                                                      )
    y_train = x_train.target
    x_train = x_train.drop('target', axis=1)
    feat_size = len(x_train.columns)
    sel_feat_size = int(feat_size * informative_pct)
    m = ensemble.RandomForestClassifier(random_state=0, n_jobs=-1)
    m.fit(x_train, y_train)
    fi, timings = get_feat_imp_model_based(imp_type, m, x_train, y_train)
    fi = fi.mean(axis=0)
#     print(fi.shape)
    #                 imp_type, model, x_train, y_train, random_state
    fi_order = nlargest(feat_size, range(len(fi)), key=lambda idx: fi[idx])
    fi_selected = fi_order[:sel_feat_size]
    result = [dataset_str, imp_type, 0, dataset_state, fi_order, fi_selected, timings]
    #                 print(result)
#     result.append(result)
    return result

def get_syn_stats_shap_based(imp_types, datasets, 
                              dataset_states, informative_pct):
    results = []
    datasets_str = list(datasets.keys())
    for j in trange(len(datasets_str), desc=f'Dataset progress'):
        try:
            for i in trange(len(dataset_states), desc=f'Processing {datasets_str[j]}'):
                for k in trange(len(imp_types)):
                    imp_type = imp_types[k] 
                    dataset_state = dataset_states[i]
                    dataset_str = datasets_str[j]
                    dataset = datasets.get(datasets_str[j])
                    result = get_shap_fi(imp_type, dataset_state, dataset_str, dataset, informative_pct)
                    results.append(result)
        except TimeoutError as e:
            print(f'skipping the execution of {dataset_str} because of {e}')
            
    columns = ['dataset', 'imp_type', 'random_state', 'data_split', 
                       'fi_order', 'fi_selected', 'timings']
    result_df = pd.DataFrame(results, columns=columns)
    return result_df

def coerce_synthethic_datasets(samples, features, percent_informative, random_state):
    data, target = make_classification(n_samples=samples, n_features=features, 
                                       n_informative=int(percent_informative*features),
                                       n_redundant=0, shuffle=False,
                                       random_state=random_state
                                      )
    feature_names = [f'x_{i}' for i in range(features)]
    # data, target, feature_names
    data_bunch = Bunch(
        data=data,
        target=target,
        feature_names=feature_names,
        informative_features=feature_names[:int(percent_informative*features)]
                        )
    return data_bunch

In [70]:
imp_types_data = [
             'mutual_info',
             'f_test', 
             'pearson', 
             'spearman', 
             'kendall', 
             'pps'
            ]
imp_types_model = [
             'gini',
             'pimp', 
            ]

n_jobs = 3

random_states = [i for i in range(1)]
dataset_states = [i for i in range(10)]

features = [10, 100]
samples = [100, 1000, 10000]
# features = [100]
# samples = [10000]
# features = [10]
# samples = [100]
synthethic_states = [i for i in range(1)]
percent_informative = 0.2
tuples = []
for i in features:
    for j in samples:
        if i != j:
            for k in synthethic_states:            
                tuples.append((i,j,k))
datasets = {}

for feat, samples, random_seed in tuples:
    datasets[f'classif_features_{feat}_samples_{samples}_seed_{random_seed}'] = coerce_synthethic_datasets(samples, feat, 0.2, random_seed)

In [60]:
res_data = get_syn_stats_data_based(imp_types_data, datasets, dataset_states, 0.2)

Dataset progress:   0%|          | 0/5 [00:00<?, ?it/s]

Processing classif_features_10_samples_100_seed_0:   0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

23.4 ms ± 3.22 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
1.66 ms ± 168 µs per loop (mean ± std. dev. of 10 runs, 1 loop each)
1.82 ms ± 317 µs per loop (mean ± std. dev. of 10 runs, 1 loop each)
2.56 ms ± 189 µs per loop (mean ± std. dev. of 10 runs, 1 loop each)
15.8 ms ± 934 µs per loop (mean ± std. dev. of 10 runs, 1 loop each)
89.9 ms ± 3 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


  0%|          | 0/6 [00:00<?, ?it/s]

...

2.3 s ± 5.52 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
8.14 ms ± 566 µs per loop (mean ± std. dev. of 10 runs, 1 loop each)
124 ms ± 360 µs per loop (mean ± std. dev. of 10 runs, 1 loop each)
337 ms ± 6.63 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
7.34 s ± 126 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
4.81 s ± 39.5 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


  0%|          | 0/6 [00:00<?, ?it/s]

2.4 s ± 4.04 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
8.23 ms ± 507 µs per loop (mean ± std. dev. of 10 runs, 1 loop each)
125 ms ± 739 µs per loop (mean ± std. dev. of 10 runs, 1 loop each)
322 ms ± 4.37 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
7.59 s ± 83.7 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
4.75 s ± 144 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


  0%|          | 0/6 [00:00<?, ?it/s]

2.39 s ± 23.7 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
8.94 ms ± 528 µs per loop (mean ± std. dev. of 10 runs, 1 loop each)
122 ms ± 752 µs per loop (mean ± std. dev. of 10 runs, 1 loop each)
316 ms ± 6.6 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
7.51 s ± 102 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
4.69 s ± 72 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [63]:
res_model = get_syn_stats_model_based(imp_types_model, datasets, dataset_states, 0.2)

Dataset progress:   0%|          | 0/5 [00:00<?, ?it/s]

Processing classif_features_10_samples_100_seed_0:   0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

17.4 ms ± 4.69 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
1.21 s ± 30.9 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


  0%|          | 0/2 [00:00<?, ?it/s]

...

17 ms ± 1.67 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
26 s ± 405 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


  0%|          | 0/2 [00:00<?, ?it/s]

16.8 ms ± 1.18 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
26 s ± 108 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


  0%|          | 0/2 [00:00<?, ?it/s]

17.1 ms ± 1.99 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
26.4 s ± 1.57 s per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [73]:
# shap is slow, need timeout sig to stop process
res_shap = get_syn_stats_shap_based(['shap'], datasets, dataset_states, 0.2)

Dataset progress:   0%|          | 0/5 [00:00<?, ?it/s]

Processing classif_features_10_samples_100_seed_0:   0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

13.8 ms ± 4.53 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

skipping the execution of classif_features_100_samples_1000_seed_0 because of Function get_shap_fi timed out after 10.0 seconds


Processing classif_features_100_samples_10000_seed_0:   0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

skipping the execution of classif_features_100_samples_10000_seed_0 because of Function get_shap_fi timed out after 10.0 seconds


In [74]:
frames = [res_data, res_model, res_shap]

In [75]:
final_df = pd.concat(frames);

In [76]:
final_df

,dataset,imp_type,random_state,data_split,fi_order,fi_selected,timings
0,classif_features_10_samples_100_seed_0,mutual_info,0,0,"[1, 5, 9, 7, 0, 2, 3, 4, 6, 8]","[1, 5]","[0.030104098999800044, 0.02757260100042913, 0...."
1,classif_features_10_samples_100_seed_0,f_test,0,0,"[1, 2, 7, 4, 3, 6, 5, 0, 9, 8]","[1, 2]","[0.0019199010002921568, 0.001570456999615999, ..."
2,classif_features_10_samples_100_seed_0,pearson,0,0,"[1, 2, 7, 4, 3, 6, 5, 0, 9, 8]","[1, 2]","[0.002370285999859334, 0.001950051000676467, 0..."
3,classif_features_10_samples_100_seed_0,spearman,0,0,"[1, 4, 2, 3, 7, 5, 6, 9, 0, 8]","[1, 4]","[0.0026753900001494912, 0.002591398000731715, ..."
4,classif_features_10_samples_100_seed_0,kendall,0,0,"[1, 4, 2, 3, 7, 5, 6, 9, 0, 8]","[1, 4]","[0.01813015900006576, 0.016762551998908748, 0...."
...,...,...,...,...,...,...,...
15,classif_features_10_samples_1000_seed_0,shap,0,5,"[1, 0, 8, 6, 3, 7, 9, 2, 4, 5]","[1, 0]","[0.4789532600007078, 0.4347276909975335, 0.433..."
16,classif_features_10_samples_1000_seed_0,shap,0,6,"[1, 0, 6, 5, 7, 4, 9, 2, 8, 3]","[1, 0]","[0.43566127400117693, 0.43838538199997856, 0.4..."
17,classif_features_10_samples_1000_seed_0,shap,0,7,"[1, 0, 6, 8, 3, 9, 5, 4, 2, 7]","[1, 0]","[0.3879009540032712, 0.3809254849984427, 0.380..."
18,classif_features_10_samples_1000_seed_0,shap,0,8,"[1, 0, 3, 5, 8, 9, 7, 2, 6, 4]","[1, 0]","[0.44764364100046805, 0.43974558300033095, 0.4..."


In [77]:
# export results to local dir as csv file
final_df.to_csv(f'classification_synthethic_data_{datetime.today().strftime("%Y-%b-%d-%H:%M:%S")}.csv',
             index=False)

In [62]:
## plots
# import seaborn as sns
# plot = sns.lineplot(x='iteration', y='mcc_test', hue='imp_type',
#             data=df_res.query('''model == "XGBClassifier"''')
#             )

# plot = sns.lineplot(x='iteration', y='mcc_test', hue='imp_type',
#             data=df_res.query('''model == "DecisionTreeClassifier"''')
#             )

# plot = sns.lineplot(x='iteration', y='mcc_test', hue='imp_type',
#             data=df_res.query('''model == "RandomForestClassifier"''')
#             )

In [250]:
# export results to local dir as csv file
# df_res.to_csv(f'classification_synthethic_data_{datetime.today().strftime("%Y-%b-%d-%H:%M:%S")}.csv',
#              index=False)